# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [90]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, make_scorer
from sklearn.linear_model import LogisticRegression

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import pickle

import warnings

warnings.filterwarnings("ignore")

In [7]:
# load data from database
engine = create_engine('sqlite:///Messages.db')
df = pd.read_sql('SELECT * FROM Messages', engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    """
    Normalize, tokenize, remove stopwords and lemmatize words
    
    params:
    text: str. Messages for preprocessing
    
    returns:
    clean_tokens: list of strings. Containing tokenize words
    """
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # Initialize Lemmatizer
    lemma = WordNetLemmatizer()
    
    lemmed = [lemma.lemmatize(w) for w in tokens if w not in stopwords.words("english")]
    
    return lemmed

In [15]:
# Test the tokenizer
text = X[0]
cleaned_tokens = tokenize(text)
print(text)
print(cleaned_tokens)

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

pipeline.fit(X_train, Y_train)

C:\Users\andre\Anaconda3\envs\data-science\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\andre\Anaconda3\envs\data-science\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\andre\Anaconda3\envs\data-science\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\andre\Anaconda3\envs\data-science\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\U

C:\Users\andre\Anaconda3\envs\data-science\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\andre\Anaconda3\envs\data-science\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [48]:
def evaluate_model(model, X, y_true, col_names):
    """
    Print evaluation metrics of the model
    
    params:
    model: sklearn models or pipelines
    X: X_train or X_test
    y_true: array. Array containing the actual label
    col_names: list of str. List containing the column names of Y labels
    
    
    """
    y_true = np.array(y_true)
    y_pred = model.predict(X)
    metrics = []
    
    for i in range(len(col_names)):
        accuracy = accuracy_score(y_true[:, i], y_pred[:, i])
        precision = precision_score(y_true[:, i], y_pred[:, i])
        recall = recall_score(y_true[:, i], y_pred[:, i])
        f1 = f1_score(y_true[:, i], y_pred[:, i])
        metrics.append([accuracy, precision, recall, f1])
    
    metrics_df = pd.DataFrame(metrics, index=col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1-score'])
    
    return metrics_df

In [49]:
col_names = list(Y.columns.values)

# Evaluation metrics for training set
train_metrics_df = evaluate_model(pipeline, X_train, Y_train, col_names)

In [50]:
# Evaluation metrics for test set
test_metrics_df = evaluate_model(pipeline, X_test, Y_test, col_names)

C:\Users\andre\Anaconda3\envs\data-science\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\andre\Anaconda3\envs\data-science\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [51]:
train_metrics_df

,Accuracy,Precision,Recall,F1-score
related,0.990472,0.992471,0.995123,0.993795
request,0.986527,0.995218,0.926409,0.959582
offer,0.998822,1.000000,0.750000,0.857143
aid_related,0.986322,0.995971,0.971151,0.983405
medical_help,0.988064,1.000000,0.848898,0.918274
medical_products,0.991394,0.996269,0.829193,0.905085
search_and_rescue,0.994775,1.000000,0.808271,0.893971
security,0.995697,0.996094,0.754438,0.858586
military,0.994672,0.996262,0.839370,0.911111
water,0.995185,0.999130,0.925121,0.960702


Most of the categories has vert low F1-score in test set. This is caused by the fact that most of the categories has labels equals to 0 except `realted` category. To improve the F-score, we may have to resample the dataset by up sampling the under represented class.

In [56]:
Y.sum() / len(Y)

related                   0.764792
request                   0.171892
offer                     0.004534
aid_related               0.417243
medical_help              0.080068
medical_products          0.050446
search_and_rescue         0.027816
security                  0.018096
military                  0.033041
water                     0.064239
food                      0.112302
shelter                   0.088904
clothing                  0.015560
money                     0.023206
missing_people            0.011449
refugees                  0.033618
death                     0.045874
other_aid                 0.132396
infrastructure_related    0.065506
transport                 0.046143
buildings                 0.051214
electricity               0.020440
tools                     0.006109
hospitals                 0.010873
shops                     0.004610
aid_centers               0.011872
other_infrastructure      0.044222
weather_related           0.280352
floods              

In [52]:
test_metrics_df

,Accuracy,Precision,Recall,F1-score
related,0.806362,0.839049,0.921628,0.878402
request,0.886584,0.772321,0.470109,0.584459
offer,0.996004,0.000000,0.000000,0.000000
aid_related,0.755187,0.758653,0.605748,0.673632
medical_help,0.917781,0.555556,0.064576,0.115702
medical_products,0.952820,0.822581,0.146974,0.249389
search_and_rescue,0.970493,0.500000,0.031250,0.058824
security,0.979253,0.250000,0.007519,0.014599
military,0.965268,0.461538,0.026667,0.050420
water,0.947902,0.858268,0.253488,0.391382


### 6. Improve your model
Use grid search to find better parameters. 

* Create a customized score function. If we use mean f1 score, the model will favour classifiers that have small number of categories that have very high f1 score, while most of the `categories` have very low f1 score. So median f1 score will give a better indicater of how well the model actually perform in this case

In [57]:
def score(y_true, y_pred):
    """
    Calculate median F1 score for multi-output labels
    
    params:
    y_true: array. Actual labels
    y_pred: array. Predicted labels return from models
    
    returns:
    f1_median: float. median f1 scores
    """
    y_true = np.array(y_true)
    f1_list = []
    for i in range(y_true.shape[1]):
        f1 = f1_score(y_true[:, i], y_pred[:, i])
        f1_list.append(f1)
    f1_median = np.median(f1_list)
    return f1_median

In [63]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [100, 200],
    'clf__estimator__min_samples_split': [2, 5, 10]}

scorer = make_scorer(score)
cv = GridSearchCV(pipeline, param_grid=parameters, scoring=scorer, verbose=1)

grid_model = cv.fit(X_train, Y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed: 447.2min finished


In [65]:
# Parameters for best model
grid_model.best_params_

{'clf__estimator__min_samples_split': 10,
 'clf__estimator__n_estimators': 100,
 'tfidf__use_idf': True,
 'vect__max_df': 1.0}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [68]:
 grid_model_df = evaluate_model(grid_model, X_test, Y_test, col_names)

With Tuning the model parameters, the median F1-score has increased by 9%. But there is still around 25% of classifiers have an F1 score near to 0.

In [75]:
grid_model_df

,Accuracy,Precision,Recall,F1-score
related,0.821884,0.832132,0.958688,0.890938
request,0.898263,0.812147,0.520833,0.634658
offer,0.996004,0.000000,0.000000,0.000000
aid_related,0.790380,0.757634,0.731393,0.744282
medical_help,0.919625,0.693878,0.062731,0.115059
medical_products,0.950669,0.882353,0.086455,0.157480
search_and_rescue,0.971108,0.833333,0.026042,0.050505
security,0.979560,0.500000,0.007519,0.014815
military,0.967573,0.791667,0.084444,0.152610
water,0.954972,0.891429,0.362791,0.515702


In [76]:
grid_model_df.describe()

,Accuracy,Precision,Recall,F1-score
count,35.000000,35.000000,35.000000,35.000000
mean,0.947577,0.622079,0.219360,0.269094
std,0.049517,0.358070,0.284069,0.300172
min,0.790380,0.000000,0.000000,0.000000
25%,0.941832,0.513158,0.011112,0.021900
50%,0.957738,0.805556,0.071429,0.133333
75%,0.980559,0.837498,0.400246,0.541351
max,0.996004,1.000000,0.958688,0.890938


In [74]:
(grid_model_df['F1-score'].mean() - test_metrics_df['F1-score'].mean()) / test_metrics_df['F1-score'].mean()

0.0939941623902584

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Logistic Regression

* We will use logistic regression in the pipeline to try to imprive the results.  Logistic regression is faster to train and a simple model, it aloows use of rich feature representation. 

In [78]:
pipeline_log = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(LogisticRegression()))
])

pipeline_log.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1.0,
                                                                    class_weight=None,
                 

In [79]:
test_log_df = evaluate_model(pipeline_log, X_test, Y_test, col_names)

In [81]:
test_log_df.describe()

,Accuracy,Precision,Recall,F1-score
count,35.000000,35.000000,35.000000,35.000000
mean,0.946985,0.632992,0.234009,0.298911
std,0.050501,0.299171,0.257880,0.276961
min,0.775012,0.000000,0.000000,0.000000
25%,0.938067,0.524194,0.035210,0.066332
50%,0.957277,0.750000,0.133858,0.225166
75%,0.981558,0.831301,0.439703,0.578159
max,0.996004,1.000000,0.961320,0.894310


In [82]:
parameters_log = {
    'vect__max_df': [1.0],
    'tfidf__use_idf': [True],
    'clf__estimator__penalty': ['l1', 'l2'],
    'clf__estimator__C': np.logspace(-5, 5, 10)}


cv_log = GridSearchCV(pipeline_log, param_grid=parameters_log, scoring=scorer, verbose=5)

grid_model_log = cv_log.fit(X_train, Y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] clf__estimator__C=1e-05, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__C=1e-05, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0, score=0.000, total= 2.6min
[CV] clf__estimator__C=1e-05, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s


[CV]  clf__estimator__C=1e-05, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0, score=0.000, total= 2.6min
[CV] clf__estimator__C=1e-05, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0 


...

[CV]  clf__estimator__C=1e-05, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0, score=0.000, total= 2.4min
[CV] clf__estimator__C=1e-05, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 10.2min remaining:    0.0s


[CV]  clf__estimator__C=1e-05, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0, score=0.000, total= 2.5min
[CV] clf__estimator__C=1e-05, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__C=1e-05, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0, score=0.000, total= 2.4min
[CV] clf__estimator__C=0.0001291549665014884, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__C=0.0001291549665014884, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0, score=0.000, total= 2.6min
[CV] clf__estimator__C=0.0001291549665014884, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__C=0.0001291549665014884, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0, score=0.000, total= 2.7min
[CV] clf__estimator__C=0.0001291549665014884, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__C=0.000129

[CV]  clf__estimator__C=46.41588833612782, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0, score=0.401, total= 5.8min
[CV] clf__estimator__C=46.41588833612782, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__C=46.41588833612782, clf__estimator__penalty=l1, tfidf__use_idf=True, vect__max_df=1.0, score=0.366, total= 5.6min
[CV] clf__estimator__C=46.41588833612782, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__C=46.41588833612782, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0, score=0.351, total= 2.5min
[CV] clf__estimator__C=46.41588833612782, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estimator__C=46.41588833612782, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0, score=0.354, total= 2.4min
[CV] clf__estimator__C=46.41588833612782, clf__estimator__penalty=l2, tfidf__use_idf=True, vect__max_df=1.0 
[CV]  clf__estim

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 162.6min finished


In [84]:
grid_model_log.best_params_

{'clf__estimator__C': 3.593813663804626,
 'clf__estimator__penalty': 'l1',
 'tfidf__use_idf': True,
 'vect__max_df': 1.0}

In [85]:
 grid_model_log_df = evaluate_model(grid_model_log, X_test, Y_test, col_names)

After performing grid search on logistic regression model, the median F1-score has increased by 44%. And it perform so much better than the previous tuned random forest model. So we shall use the tuned Logistic regression model.

In [87]:
grid_model_log_df.describe()

,Accuracy,Precision,Recall,F1-score
count,35.000000,35.000000,35.000000,35.000000
mean,0.946673,0.574467,0.365502,0.431982
std,0.053634,0.224811,0.246051,0.245787
min,0.759951,0.000000,0.000000,0.000000
25%,0.943061,0.471459,0.189205,0.275716
50%,0.959428,0.625000,0.327381,0.416510
75%,0.983556,0.736281,0.577411,0.650373
max,0.995851,0.883810,0.916970,0.887582


In [88]:
(grid_model_log_df['F1-score'].mean() - test_log_df['F1-score'].mean()) / test_log_df['F1-score'].mean()

0.4451833323502813

### 9. Export your model as a pickle file

In [92]:
with open('models/clf.pkl', 'wb') as out_file:
    pickle.dump(grid_model_log, out_file )

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.